In [1]:
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer, OneHotEncoder, StandardScaler

### DataLoader

In [2]:
def DataPreprocessor(source,
                     target=None,
                     ignore=None,
                     random_state=42):
    
    df = pd.read_csv(source)
    
    target = target
    ignore = ignore

    inputs = sorted(set(df.columns) - set(target) - set(ignore))

    # Data type detection
    numerical_ix = df[inputs].select_dtypes(include=['int64', 'float64']).columns
    categorical_ix = df[inputs].select_dtypes(include=['object', 'bool']).columns
    
    # Data transforms
    cat_transform = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
        ('oh_encoder', OneHotEncoder(sparse=False))
    ])
    num_transform = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    transform_x = ColumnTransformer(transformers=[
        ('cat', cat_transform, categorical_ix),
        ('num', num_transform, numerical_ix)
    ])
    transform_y = ColumnTransformer(transformers=[
        ('num', Normalizer(), pd.Index(target))
    ])
    
    trans_x = transform_x.fit_transform(df[inputs])
    trans_y = transform_y.fit_transform(df)
    
    return train_test_split(trans_x, trans_y, test_size=0.2, random_state=random_state)

In [3]:
source = "../../data/titanic/titanic_train.csv"
target = ["Survived"]
ignore = ["Name", "Cabin", "Ticket"]

x_train, x_test, y_train, y_test = DataPreprocessor(source, target=target, ignore=ignore)

### MLP Model

In [123]:
# mlp_params

# layer structure
hidden_layers = [1, 2, 3, 4]
nodes = [16, 32, 64, 128, 256]

# compile params
learning_rate = [0.01, 0.02, 0.1, 0.2]
optimizers = [keras.optimizers.Adadelta, keras.optimizers.Adagrad(),
              keras.optimizers.Adam(), keras.optimizers.Adamax(),
              keras.optimizers.RMSprop(), keras.optimizers.SGD()]
losses = {'reg': [keras.losses.MSE, keras.losses.MAE,keras.losses.MSLE],
          'cls': [keras.losses.binary_crossentropy],
          'bi_cls': [keras.losses.categorical_crossentropy]}
metrics = {'reg': ['mse', 'mae'],
           'cls': ['accuracy']}

In [126]:
def build_model(x_train):
    nn_inputs = keras.Input(shape=(x_train.shape[1],), name='input')
    x = keras.layers.Dense(16, activation='relu', name='dense_1')(nn_inputs)
    nn_outputs = keras.layers.Dense(1, activation='sigmoid', name='prediction')(x)
    model = keras.Model(inputs=nn_inputs, outputs=nn_outputs)
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    tf.random.set_seed(42)    
    model.fit(x_train, y_train, epochs=5, batch_size=32)

In [127]:
build_model(x_train)

Train on 837 samples
Epoch 1/5
837/837 [==============================] - 0s 216us/sample - loss: 0.6637 - accuracy: 0.6834
Epoch 2/5
837/837 [==============================] - 0s 26us/sample - loss: 0.6052 - accuracy: 0.7527
Epoch 3/5
837/837 [==============================] - 0s 26us/sample - loss: 0.5585 - accuracy: 0.7634
Epoch 4/5
837/837 [==============================] - 0s 25us/sample - loss: 0.5189 - accuracy: 0.7778
Epoch 5/5
837/837 [==============================] - 0s 25us/sample - loss: 0.4841 - accuracy: 0.8088


In [87]:
class MLP(tf.keras.Model):
    
    def __init__(self):
        super(MLP, self).__init__()
        self.dense1 = keras.layers.Dense(16, activation='relu')
        self.output_layer = keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        return self.output_layer(x)

In [88]:
model = MLP()

In [89]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [90]:
tf.random.set_seed(42)
model.fit(x_train, y_train, epochs=5, batch_size=32)

Train on 837 samples
Epoch 1/5
837/837 [==============================] - 0s 218us/sample - loss: 0.6949 - accuracy: 0.4779
Epoch 2/5
837/837 [==============================] - 0s 25us/sample - loss: 0.6472 - accuracy: 0.7407
Epoch 3/5
837/837 [==============================] - 0s 25us/sample - loss: 0.5988 - accuracy: 0.7766
Epoch 4/5
837/837 [==============================] - 0s 24us/sample - loss: 0.5489 - accuracy: 0.7838
Epoch 5/5
837/837 [==============================] - 0s 26us/sample - loss: 0.5060 - accuracy: 0.8088


In [ ]:
class MLP2(tf.keras.Model):
    
    def __init__(self, num_layers=[1]):
        super(MLP2, self).__init__()
        for i in num_layers:
            self.dense = 
        self.dense1 = keras.layers.Dense(16, activation='relu')
        self.output_layer = keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        return self.output_layer(x)

In [94]:
num_layers = [1, 2, 3, 4]

In [101]:
model = keras.Sequential()
model.add(keras.layers.Dense(16, input_shape=(x_train.shape[1],)))
for i in num_layers:
    model.add(keras.layers.Dense(16))

In [103]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [104]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 16)                12496     
_________________________________________________________________
dense_16 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_17 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_18 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_19 (Dense)             (None, 16)                272       
Total params: 13,584
Trainable params: 13,584
Non-trainable params: 0
_________________________________________________________________


In [303]:
hidden_layers = [1, 2, 3]
nodes = [16, 32, 64, 128, 256]
grid = [hidden_layers, nodes]

models = []

for param_tuple in itertools.product(*grid):
    params = {'hidden_layers': param_tuple[0],
              'nodes': param_tuple[1]}
    
    model = keras.Sequential()
    model.add(keras.layers.Dense(10, input_shape=(x_train.shape[1],)))
    
    for _ in range(params['hidden_layers']):
        model.add(keras.layers.Dense(params['nodes'], activation='relu'))
        
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    models.append(model)

In [305]:
len(models)

15

In [304]:
for model in models:
    model.summary()

Model: "sequential_77"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3238 (Dense)           (None, 10)                7810      
_________________________________________________________________
dense_3239 (Dense)           (None, 16)                176       
_________________________________________________________________
dense_3240 (Dense)           (None, 1)                 17        
Total params: 8,003
Trainable params: 8,003
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3241 (Dense)           (None, 10)                7810      
_________________________________________________________________
dense_3242 (Dense)           (None, 32)                352       
____________________________

In [281]:
def create_model(hidden_layer=1, node=16):
    model = keras.Sequential()
    model.add(keras.layers.Dense(10, input_shape=(x_train.shape[1],)))
    
    for i in range(hidden_layer):
        model.add(keras.layers.Dense(node, activation='relu'))
        
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [282]:
model = create_model(hidden_layer=1, node=16)

In [283]:
model

In [284]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3015 (Dense)           (None, 10)                7810      
_________________________________________________________________
dense_3016 (Dense)           (None, 16)                176       
_________________________________________________________________
dense_3017 (Dense)           (None, 1)                 17        
Total params: 8,003
Trainable params: 8,003
Non-trainable params: 0
_________________________________________________________________


In [300]:
nodes = [16, 32, 64, 128, 256]
models = []
hidden_layers = 3
for hidden in range(hidden_layers):
    for node in nodes:
        models.append(create_model(hidden_layer=hidden+1, node=node))

In [301]:
len(models)

15

In [302]:
for model in models:
    model.summary()

Model: "sequential_62"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3178 (Dense)           (None, 10)                7810      
_________________________________________________________________
dense_3179 (Dense)           (None, 16)                176       
_________________________________________________________________
dense_3180 (Dense)           (None, 1)                 17        
Total params: 8,003
Trainable params: 8,003
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3181 (Dense)           (None, 10)                7810      
_________________________________________________________________
dense_3182 (Dense)           (None, 32)                352       
____________________________